# Explore BBC Monitoring Corpus
Felix Zaussinger | 08.01.2021

## Core Analysis Goal(s)
1. Discover
2. Descriptive statistics
3. Feature extraction

## Key Insight(s)
1.
2.
3.

In [22]:
# magic commands
%load_ext autoreload
%autoreload 2

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# imports
import os
import re
import sys
import glob
import textacy
import logging
import numpy as np
import pandas as pd
from tqdm import tqdm
import en_core_web_lg
import seaborn as sns
from pathlib import Path
from textacy import preprocessing
from dotenv import find_dotenv, load_dotenv

# module settings
sns.set_context("poster")
sns.set(rc={'figure.figsize': (16, 9.)})
sns.set_style("ticks")

# logging
logging.basicConfig(level=logging.INFO, stream=sys.stdout)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Define directory structure

In [23]:
# project directory
abspath = os.path.abspath('')
project_dir = str(Path(abspath).parents[0])

# sub-directories
data_raw = os.path.join(project_dir, "data", "raw")
data_interim = os.path.join(project_dir, "data", "interim")
data_processed = os.path.join(project_dir, "data", "processed")
figure_dir = os.path.join(project_dir, "reports", "figures")

Load and configure spacy nlp model

In [24]:
nlp = en_core_web_lg.load()
nlp.max_length = 30000000
nlp.Defaults.stop_words |= {
    "hyperlink",
    "HYPERLINK",
}

Load textacy corpus that stores the pre-processed BBC Monitoring data and
its metadata

In [25]:
fname_corpus = "BBC_2007_07_04_CORPUS_TEXTACY.bin.gz"
corpus = textacy.Corpus.load(
    lang=nlp,
    filepath=os.path.join(data_processed, fname_corpus),
    store_user_data=True
)

Inspect corpus

In [30]:
corpus

Vectorization